In [1]:
!pip install -q ftfy
!pip install transformers -q
!pip install jsonlines


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.chdir('/content/drive/MyDrive/doutorado/P_IA368DD_2023S/aula6')

import gc

In [3]:
import torch
from torch.utils.data import DataLoader
from torch.nn import functional as F

from transformers import AutoTokenizer,  AutoModel, BatchEncoding, get_cosine_with_hard_restarts_schedule_with_warmup

In [4]:
!export PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.6

In [5]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import ftfy
import jsonlines

from sklearn.model_selection import train_test_split

In [6]:
TRAIN_OUTPUT_FOLDER = "./checkpoints/"

path_data = "./data/"
if not os.path.exists(f"{path_data}"):
    os.mkdir(path_data)
    !wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
    !mv msmarco_triples.train.tiny.tsv {path_data}
    print("Pasta criada!")
else:
    print("Pasta já existe")

Pasta já existe


In [7]:
df = pd.read_csv(f"{path_data}/msmarco_triples.train.tiny.tsv", delimiter="\t", 
                 header=None, names=["query", "positive", "negative"])
df.head(2)

,query,positive,negative
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."


In [10]:
df = df.drop(columns=["negative"])
df['query'] = df['query'].apply(ftfy.fix_text)
df['positive'] = df['positive'].apply(ftfy.fix_text)

In [11]:
train_df, test_df = train_test_split(df,test_size=1000,random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [27]:
class MyCustomDprDataset(torch.utils.data.Dataset):
  def __init__(self, data,tokenizer,type_="both"):
      self.passages = []
      self.queries = []
      self.type_ = type_

      if type_ == "query" or type_ == "both":
        self.queries = data["query"]
      if type_ == "corpus" or type_ == "both":
        self.passages = data["positive"]
      #self.passages = data["positive"]
      #self.queries = data["query"]
      
      self.tokenizer = tokenizer
      self.tokenized_passages = None
      self.tokenized_queries = None

  def __len__(self):
    if self.type_ == "query":
      return len(self.queries)
    else:
      return len(self.passages)

  def __getitem__(self, idx):
    if self.type_ == "query":
      queries  = self.tokenizer(self.queries[idx], return_length=False)
      return {'query' : queries}
    elif self.type_ == "corpus":
      passages = self.tokenizer(self.passages[idx],return_length=False)
      return {'passage':passages}
    else:
      passages = self.tokenizer(self.passages[idx],return_length=False)
      queries  = self.tokenizer(self.queries[idx], return_length=False)
      return {'passage': passages,
             'query' : queries
            }

In [13]:
%%markdown
# load models

# load models


In [8]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
MODEL_NAME = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [14]:


passage_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
queries_model = AutoModel.from_pretrained(MODEL_NAME).to(device)

In [15]:

%%markdown
# Load dataset

# Load dataset


In [16]:
%%markdown
### load dummy dataset to check if ok

### load dummy dataset to check if ok


In [18]:
dummy_dataset = MyCustomDprDataset(df.iloc[:10], tokenizer=tokenizer)

In [10]:
def collate_fn_passage(batch):
    passages = [item["passage"] for item in batch]
    return BatchEncoding(tokenizer.pad(passages, return_tensors='pt'))
def collate_fn_query(batch):
    passages = [item["query"] for item in batch]
    return BatchEncoding(tokenizer.pad(passages, return_tensors='pt'))

In [ ]:

dummy_loader = DataLoader(dummy_dataset, batch_size=4, shuffle=True, num_workers=0,collate_fn=collate_fn_passage)
first_batch = next(iter(dummy_loader))
print(first_batch)

In [21]:
%%markdown
## Load actuals datasets

## Load actuals datasets


In [17]:
train_dataset = MyCustomDprDataset(train_df, tokenizer=tokenizer)
eval_dataset = MyCustomDprDataset(test_df, tokenizer=tokenizer)

In [20]:
train_passages_dataloader = DataLoader(train_dataset,
                                       batch_size=4,
                                       shuffle=False,
                                       num_workers=0,
                                       collate_fn=collate_fn_passage)

train_queries_dataloader = DataLoader(train_dataset, 
                                    batch_size=4, 
                                    shuffle=False, 
                                    collate_fn=collate_fn_query)

eval_passages_dataloader = DataLoader(eval_dataset,
                                      batch_size=4, 
                                      shuffle=False, 
                                      collate_fn=collate_fn_passage)

eval_queries_dataloader = DataLoader(eval_dataset, 
                                    batch_size=4, 
                                    shuffle=False, 
                                    collate_fn=collate_fn_query)
     

In [25]:
%%markdown
# auxiliar functions

# auxiliar functions


In [24]:
def cross_entropy(passages_outputs, queries_outputs):
  
  passages_cls = passages_outputs.last_hidden_state[:, 0, :]
  queries_cls = queries_outputs.last_hidden_state[:, 0, :]
  
  
  loss = F.cross_entropy(queries_cls,passages_cls)
  return loss

def compute_loss(passages_outputs, queries_outputs):

    passages_cls = passages_outputs.last_hidden_state[:, 0, :]
    queries_cls = queries_outputs.last_hidden_state[:, 0, :]

    all_passages_all_queries_dot_product = torch.mm(passages_cls, queries_cls.t())
    passages_and_positive_queries = all_passages_all_queries_dot_product.diag().unsqueeze(1)

    dot_product_differences = all_passages_all_queries_dot_product - passages_and_positive_queries

    loss = torch.log(torch.sum(torch.exp(dot_product_differences), dim=1))

    non_zeroed_losses = (loss > 1e-8).float().sum()

    if non_zeroed_losses > 0.0:
        final_loss = torch.sum(loss) / non_zeroed_losses
    else:
        final_loss = torch.mean(loss)

    return final_loss

#labels = torch.tensor([1,0]).unsqueeze(0)
#outputs = model(input_ids, attention_mask=attention_mask)
def evaluate(device,
             passages_model, 
             queries_model, 
             passages_dataloader, 
             queries_dataloader, 
             #min_eval_loss,
             current_epoch=0):
    global min_eval_loss
    
    eval_losses = []

    passages_model.eval()
    queries_model.eval()

    with torch.no_grad():
        for batch in tqdm(list(zip(passages_dataloader, queries_dataloader)), mininterval=0.5, desc="Eval", disable=False):
            
            passages_outputs = passages_model(**batch[0].to(device))
            queries_outputs = queries_model(**batch[1].to(device))

            eval_losses.append(compute_loss(passages_outputs, queries_outputs).cpu().numpy())

    final_loss = np.mean(eval_losses)

    print("Eval loss: {:0.3f}".format(final_loss))

    if final_loss < min_eval_loss:
        #print("New minimal validation loss; saving model...")

        #training_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        checkpoint_name = "checkpoint_epoch_{}_loss_{}".format(current_epoch, final_loss)
        passages_model.save_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, checkpoint_name, "_passages"))
        queries_model.save_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, checkpoint_name, "_topics"))

        #min_eval_loss['checkpoint_name'] = checkpoint_name
        min_eval_loss = final_loss

In [22]:
%%markdown
## Evaluate model with no fine-tuning

## Evaluate model with no fine-tuning


In [25]:
min_eval_loss = 999999
evaluate(device,
        passages_model=passage_model, 
        queries_model=queries_model,
        passages_dataloader=eval_passages_dataloader,
        queries_dataloader=eval_queries_dataloader,
        current_epoch=-1)

Eval:   0%|          | 0/250 [00:00<?, ?it/s]

Eval loss: 1.379


In [26]:
N_EPOCHS = 5

In [27]:
passages_optimizer = torch.optim.AdamW(passage_model.parameters(), lr=1e-4)
passages_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(passages_optimizer,
                                                                        0,
                                                                        N_EPOCHS * len(train_passages_dataloader),
                                                                        1)


queries_optimizer = torch.optim.AdamW(queries_model.parameters(), lr=1e-4)
queries_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(passages_optimizer, 
                                                                        0, 
                                                                        N_EPOCHS * len(train_queries_dataloader),
                                                                        1)

In [28]:
current_training_step = 0
min_eval_loss = 99999

for epoch in tqdm(range(N_EPOCHS), desc='Epochs'):
    
    passage_model.train()
    queries_model.train()

    train_losses = []
    
    tqdm_batches = tqdm(list(zip(train_passages_dataloader, train_queries_dataloader)), mininterval=0.5, desc='Train', disable=False)
    for batch in tqdm_batches:

        current_training_step += 1

        passages_optimizer.zero_grad()
        queries_optimizer.zero_grad()
        
        passages_outputs = passage_model(**batch[0].to(device))
        queries_outputs = queries_model(**batch[1].to(device))

        final_loss = compute_loss(passages_outputs, queries_outputs)

        final_loss.backward()

        passages_optimizer.step()
        queries_optimizer.step()

        passages_scheduler.step()
        queries_scheduler.step()

        train_losses.append(final_loss.detach().cpu().numpy())

        tqdm_batches.set_description("Loss {:0.4f}".format(train_losses[-1]))

        #if LINK_WITH_COMET:
        #    experiment.log_metrics({'train loss': train_losses[-1],
        #                            'learning_rate': passages_scheduler.get_last_lr()},
        #                            step=current_training_step)

    print("Epoch: {}, Training loss: {:0.4f}".format(epoch + 1, np.mean(train_losses)))
    
    #if LINK_WITH_COMET:
    #    experiment.log_metrics({'train loss': np.mean(train_losses)},
    #                           epoch=epoch)


    evaluate(device,
             passages_model=passage_model, 
             queries_model=queries_model,
             passages_dataloader=eval_passages_dataloader,
             queries_dataloader=eval_queries_dataloader,
             #min_eval_loss=min_eval_loss,
             current_epoch=epoch)
    
    #train_dataset.shuffle()

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch: 1, Training loss: 0.6351


Eval:   0%|          | 0/250 [00:00<?, ?it/s]

Eval loss: 0.432


...

Eval loss: 0.293


Train:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch: 5, Training loss: 0.1082


Eval:   0%|          | 0/250 [00:00<?, ?it/s]

Eval loss: 0.291


In [2]:
%%markdown
#Some Experiments
#Queries download

#Some Experiments
#Queries download


In [9]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
!mv test.tsv data/trec-covid

--2023-04-19 23:22:48--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 13.35.7.28, 13.35.7.13, 13.35.7.102, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  --.-KB/s    in 0.04s   

2023-04-19 23:22:48 (20.8 MB/s) - ‘test.tsv’ saved [980831/980831]



In [11]:
query_model = AutoModel.from_pretrained(os.path.join("fine_tuned/checkpoints_seiti_loss", "checkpoint_epoch_2_loss_0.2808033227920532", "_topics")).to(device)
doc_model = AutoModel.from_pretrained(os.path.join("fine_tuned/checkpoints_seiti_loss", "checkpoint_epoch_2_loss_0.2808033227920532", "_passages")).to(device)

In [28]:
query_ids = []
query_texts = []

with jsonlines.open(f"data/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)

query_texts = pd.DataFrame(query_texts,columns=["query"])
#test_queries_tokenized = tokenizer(query_texts, max_length=max_length, 
#                                   truncation=True, padding=True)
dataset_queries_test = MyCustomDprDataset(query_texts,tokenizer,"query")
test_queries_dataloader = DataLoader(dataset_queries_test,
                                       batch_size=4,
                                       shuffle=False,
                                       num_workers=0,
                                       collate_fn=collate_fn_query)
#dataloader_queries_test = data.DataLoader(dataset_queries_test, 
#                                          #batch_size=batch_size, shuffle=False, 
#                                          collate_fn=collate_fn)



In [29]:
passage_ids = []
passage_texts = []

with jsonlines.open(f"data/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)

passage_texts = pd.DataFrame(passage_texts,columns=["positive"])
#test_queries_tokenized = tokenizer(query_texts, max_length=max_length, 
#                                   truncation=True, padding=True)
dataset_passage_test = MyCustomDprDataset(passage_texts,tokenizer,"corpus")
test_passages_dataloader = DataLoader(dataset_passage_test,
                                       batch_size=4,
                                       shuffle=False,
                                       num_workers=0,
                                       collate_fn=collate_fn_passage)

In [30]:
query_model.eval()

queries_matrix = None
#print('here')
with torch.no_grad():
  #print('here')
  for query_batch in tqdm(test_queries_dataloader, mininterval=0.5, desc='Test', 
                                          disable=False):
    #print('here')
    query_outputs = query_model(**query_batch.to(device))
    query_cls = query_outputs.last_hidden_state[:, 0, :]

    if queries_matrix is None:
      queries_matrix = query_cls
    else:
      queries_matrix = torch.cat((queries_matrix, query_cls), dim=0)

Test:   0%|          | 0/13 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [1]:
doc_model.eval()

passages_matrix = None

with torch.no_grad():
  for doc_batch in tqdm(test_passages_dataloader, mininterval=0.5, desc='Test', 
                                          disable=False):
    doc_outputs = doc_model(**doc_batch.to(device))
    doc_cls = doc_outputs.last_hidden_state[:, 0, :]

    if passages_matrix is None:
      passages_matrix = doc_cls
    else:
      passages_matrix = torch.cat((passages_matrix, doc_cls), dim=0)

NameError: ignored